In [5]:
import pandas as pd

from src.preprocess import fast_clean_text, parallel_apply, process_texts

column_names = ["sentiment", "id", "date", "query", "user", "text"]

df = pd.read_csv(r'C:\Users\User\PycharmProjects\sentiment\data\training.1600000.processed.noemoticon.csv', encoding='latin-1', names=column_names, header=None)
print(df.head())

   sentiment          id                          date     query  \
0          0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1          0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2          0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3          0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4          0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [6]:
print(df["sentiment"].value_counts())

sentiment
0    800000
4    800000
Name: count, dtype: int64


In [30]:
df1 = pd.read_csv(r'C:\Users\User\PycharmProjects\sentiment\data\Twitter_Data.csv', encoding='latin-1')
df1.rename(columns={'category':'sentiment'}, inplace=True)
df1.rename(columns={'clean_text':'text'}, inplace=True)
print(df1['sentiment'].value_counts())
#df1 = df1[df1['sentiment'] != 'irrelevant']
df1['sentiment'] = df1['sentiment'].map({-1:0,0:2,1:4})
print(df1.head())

sentiment
 1.0    72250
 0.0    55213
-1.0    35510
Name: count, dtype: int64
                                                text  sentiment
0  when modi promised âminimum government maxim...        0.0
1  talk all the nonsense and continue all the dra...        2.0
2  what did just say vote for modi  welcome bjp t...        4.0
3  asking his supporters prefix chowkidar their n...        4.0
4  answer who among these the most powerful world...        4.0


In [31]:
df_combined = pd.concat([df[['text','sentiment']], df1[['text', 'sentiment']]] , ignore_index=True)
print(df_combined['sentiment'].value_counts())
#df_combined.to_csv("sentiment_combined.csv", index=False)


sentiment
4.0    872250
0.0    835510
2.0     55213
Name: count, dtype: int64


In [40]:

df_combined['text'] = df_combined['text'].astype(str)
df_combined['text'] = df_combined['text'].fillna("")
print(df_combined[~df_combined['text'].apply(lambda x: isinstance(x, str))])



Empty DataFrame
Columns: [text, sentiment]
Index: []


In [53]:
from src.preprocess import  parallel_apply, process_texts
import multiprocessing as mp
import os

num_cores=mp.cpu_count() // 2
df_combined = parallel_apply(df_combined,process_texts, num_cores)
save_path = os.path.join(os.getcwd(), "data", "clean_tweets.csv")

#df_combined = parallel_apply(df_combined, process_texts)
#df_combined['text']=df_combined['text'].swifter.apply(fast_clean_text)
df_combined.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv", index=False)

   02401  Borderlands  Positive  \
0   2401  Borderlands  Positive   
1   2401  Borderlands  Positive   
2   2401  Borderlands  Positive   
3   2401  Borderlands  Positive   
4   2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     


In [25]:
import pandas as pd
df_clean_tweets = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv")  # Plik bazowy
df_final_fixed = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\Final_Fixed_Sentiment_CSV.csv")
df_filtered = df_final_fixed[df_final_fixed["sentiment"] == 2]
df_combined = pd.concat([df_clean_tweets, df_filtered], ignore_index=True)
print(df_combined['sentiment'].value_counts())
df_combined.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv", index=False)



sentiment
4.0    872250
0.0    835510
2.0     67864
Name: count, dtype: int64


In [23]:
df_next = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\YoutubeCommentsDataSet.csv") 
sentiment_mapping = {
    "negative": 0,
    "neutral": 2,
    "positive": 4
}
df_next["Sentiment"] = df_next["Sentiment"].map(sentiment_mapping)
df_next = df_next.rename(columns={"Comment":"text"})
df_next = df_next.rename(columns={"Sentiment":"sentiment"})
print(df_next['sentiment'].value_counts())
df_filtr= df_next[df_next["sentiment"] ==2]
df_comb = pd.concat([df_clean_tweets,df_filtr], ignore_index=True)
print(df_comb['sentiment'].value_counts())
df_comb.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv", index=False)




sentiment
4    11432
2     4638
0     2338
Name: count, dtype: int64
sentiment
4.0    872250
0.0    835510
2.0     59851
Name: count, dtype: int64


In [39]:
s = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv") 
print(s.head())
print(s['sentiment'].value_counts())

                                                text  sentiment
0  thats bummer you shoulda got david carr third ...        0.0
1  upset he cant update his facebook texting migh...        0.0
2  i dived many times ball managed save rest go o...        0.0
3            my whole body feels itchy like its fire        0.0
4  no its not behaving all im mad why am i here b...        0.0
sentiment
4.0    872250
0.0    835510
2.0     67864
Name: count, dtype: int64


In [48]:
last = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\neutral_tweets.csv") 
last["sentiment"] =2
last.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\neutral_tweets.csv", index=False)
print(last.tail())
print(last["sentiment"].value_counts())

       Unnamed: 0                                               text  \
20668       20668  overexposed tour maroon cab through eyes drumm...   
20669       20669                   wait e news just said its sunday   
20670       20670  if only possible part blackpool illuminations ...   
20671       20671  no im hilton head till th lol go jason aldean ...   
20672       20672  washington reuters us vice president joe biden...   

       sentiment  
20668          2  
20669          2  
20670          2  
20671          2  
20672          2  
sentiment
2    20673
Name: count, dtype: int64


In [49]:
from src.preprocess import  parallel_apply, process_texts, fast_clean_text
import multiprocessing as mp
import swifter
import os

last["text"] = last["text"].fillna("").astype(str)

num_cores=mp.cpu_count() // 2
last = parallel_apply(last,process_texts, num_cores)
#save_path = os.path.join(os.getcwd(), "data", "clean_tweets.csv")

#df_combined = parallel_apply(df_combined, process_texts)
last["text"] = last["text"].apply(fast_clean_text)
last.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\neutral_tweets.csv", index=False)

print(last["text"].dtype)
print(last["text"].head(10))  
print(last["sentiment"].value_counts())



object
0    ben smith smith concussion remains out lineup ...
1    sorry bout stream last night i crashed out wil...
2    chase headleys rbi double th inning off david ...
3    cena aj sitting tree kissing st goes ajs job t...
4    well said hmw can you now address why texans f...
5    just said hello dennis kucinich he walked casu...
6    tickets sam smith concert sept cheap flew our ...
7    fiorentina have reportedly opened talks chelse...
8    what jamie foxx doing sitting next niall like ...
9    st uneasyness when he got v defensive about ch...
Name: text, dtype: object
sentiment
2    20673
Name: count, dtype: int64


In [50]:
ostatni = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\neutral_tweets.csv", index_col=0)
ostatni.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\neutral_tweets.csv", index=False)

In [51]:
print(ostatni.head())

                                                         text  sentiment
Unnamed: 0                                                              
0           ben smith smith concussion remains out lineup ...          2
1           sorry bout stream last night i crashed out wil...          2
2           chase headleys rbi double th inning off david ...          2
3           cena aj sitting tree kissing st goes ajs job t...          2
4           well said hmw can you now address why texans f...          2


In [53]:
df_comb_final = pd.concat([df_clean_tweets,ostatni], ignore_index=True)
print(df_comb_final['sentiment'].value_counts())
df_comb_final.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\clean_tweets_final.csv", index=False)


sentiment
4.0    872250
0.0    835510
2.0     80524
Name: count, dtype: int64


In [58]:
counts = df_comb_final['sentiment'].value_counts()
print(counts)
min_count = counts.min()

df_0 = df_comb_final[df_comb_final["sentiment"] == 0.0].sample(n=min_count, random_state=42)
df_2 = df_comb_final[df_comb_final["sentiment"] == 2.0]  # Wszystkie wiersze, bo to najmniejsza klasa
df_4 = df_comb_final[df_comb_final["sentiment"] == 4.0].sample(n=min_count, random_state=42)

df_balanced = pd.concat([df_0, df_2, df_4])
print(df_balanced.head())
print(df_balanced['sentiment'].value_counts())
df_balanced = df_balanced.reset_index(drop=True)

df_balanced.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv", index=False)

sentiment
4.0    872250
0.0    835510
2.0     80524
Name: count, dtype: int64
                                                      text  sentiment
629815                   sad about results tonights sytycd        0.0
80733    oooh happy th then hehe errr ya i want cooking...        0.0
1685215  who announced st nuclear test indira gandhi wh...        0.0
229664   cavs its over done magic ampamp penguins ftw b...        0.0
522547   they didnt have mojo classic britpop only mojo...        0.0
sentiment
0.0    80524
2.0    80524
4.0    80524
Name: count, dtype: int64


In [1]:
import pandas as pd
df_balanced = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv")
print(df_balanced['sentiment'].value_counts())
print(df_balanced.head())

sentiment
0.0    80524
2.0    80524
4.0    80524
Name: count, dtype: int64
                                                text  sentiment
0                  sad about results tonights sytycd        0.0
1  oooh happy th then hehe errr ya i want cooking...        0.0
2  who announced st nuclear test indira gandhi wh...        0.0
3  cavs its over done magic ampamp penguins ftw b...        0.0
4  they didnt have mojo classic britpop only mojo...        0.0


In [8]:
print("🔍 Przed czyszczeniem:", df_balanced.shape[0], "wierszy")

df_balanced = df_balanced[df_balanced["text"].str.strip() != ""]
df_balanced = df_balanced.dropna(subset=["text"])

print("Po czyszczeniu:", df_balanced.shape[0], "wierszy")


print(df_balanced["text"].dtype)
print(df_balanced["text"].head(10))  # Wyświetl pierwsze wartości w kolumnie "text"
print(df_balanced["sentiment"].value_counts())
df_balanced.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv", index=False)


🔍 Przed czyszczeniem: 240951 wierszy
✅ Po czyszczeniu: 240951 wierszy
object
0                    sad about results tonights sytycd
1    oooh happy th then hehe errr ya i want cooking...
2    who announced st nuclear test indira gandhi wh...
3    cavs its over done magic ampamp penguins ftw b...
4    they didnt have mojo classic britpop only mojo...
5    twittering looked house today made offer saw b...
6    racing i flatted half way through rejoined bun...
7    possibly worst site i have ever seen i suck we...
8    expecting my mbp back morning applecare had le...
9    i read quotflquot florida btw i expected you s...
Name: text, dtype: object
sentiment
0.0    80357
4.0    80302
2.0    80292
Name: count, dtype: int64


In [9]:
from src.preprocess import  parallel_apply, process_texts, fast_clean_text
import multiprocessing as mp
import swifter
import os

#df_balanced["text"] = last["text"].fillna("").astype(str)

num_cores=mp.cpu_count() // 2
df_balanced = parallel_apply(df_balanced,process_texts, num_cores)
#save_path = os.path.join(os.getcwd(), "data", "clean_tweets.csv")

#df_combined = parallel_apply(df_combined, process_texts)
df_balanced["text"] = df_balanced["text"].apply(fast_clean_text)
df_balanced.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced4Ml.csv", index=False)

print(df_balanced["text"].dtype)
print(df_balanced["text"].head(10))  # Wyświetl pierwsze wartości w kolumnie "text"
print(df_balanced["sentiment"].value_counts())


object
0                    sad about results tonights sytycd
1    oooh happy th then hehe errr ya want cooking l...
2    who announced st nuclear test indira gandhi wh...
3    cavs its over done magic ampamp penguins ftw b...
4    they didnt mojo classic britpop only mojo king...
5    twittering looked house today made offer saw b...
6    racing flatted half way through rejoined bunch...
7    possibly worst site ever seen suck web design ...
8    expecting mbp back morning applecare had less ...
9    read quotflquot florida btw expected say quotn...
Name: text, dtype: object
sentiment
0.0    80357
4.0    80302
2.0    80292
Name: count, dtype: int64


In [11]:
print("🔍 Przed czyszczeniem:", df_balanced.shape[0], "wierszy")

df_balanced = df_balanced[df_balanced["text"].str.strip() != ""]
df_balanced = df_balanced.dropna(subset=["text"])

print("Po czyszczeniu:", df_balanced.shape[0], "wierszy")


print(df_balanced["text"].dtype)
print(df_balanced["text"].head(10))  # Wyświetl pierwsze wartości w kolumnie "text"
print(df_balanced["sentiment"].value_counts())
df_balanced.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced4ML.csv", index=False)


🔍 Przed czyszczeniem: 240925 wierszy
✅ Po czyszczeniu: 240925 wierszy
object
0                    sad about results tonights sytycd
1    oooh happy th then hehe errr ya want cooking l...
2    who announced st nuclear test indira gandhi wh...
3    cavs its over done magic ampamp penguins ftw b...
4    they didnt mojo classic britpop only mojo king...
5    twittering looked house today made offer saw b...
6    racing flatted half way through rejoined bunch...
7    possibly worst site ever seen suck web design ...
8    expecting mbp back morning applecare had less ...
9    read quotflquot florida btw expected say quotn...
Name: text, dtype: object
sentiment
0.0    80351
2.0    80289
4.0    80285
Name: count, dtype: int64


In [41]:

# Zapisujemy oczyszczony plik (opcjonalnie)
df6= pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv")
df_filtro = df6[~df6["text"].str.contains(r"\bmodi\b", case=False, na=False)]

original_len = len(df6)

removed_rows = original_len - len(df_filtro)
print(f"Usunięto {removed_rows} wierszy zawierających 'modi'.")

# Podgląd danych


print(df_filtro['sentiment'].value_counts())
df.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv",index=False)


Usunięto 58 wierszy zawierających 'modi'.
sentiment
4.0    33832
2.0    19801
0.0    12480
Name: count, dtype: int64


In [31]:
csv_files = [r"C:\Users\User\PycharmProjects\sentiment\data\1.csv", r"C:\Users\User\PycharmProjects\sentiment\data\2.csv", r"C:\Users\User\PycharmProjects\sentiment\data\3.csv", r"C:\Users\User\PycharmProjects\sentiment\data\4.csv"]
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
print(df['sentiment'].value_counts())
df.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\combined1234.csv",index=False)



sentiment
4.0    33844
2.0    19821
0.0    12482
Name: count, dtype: int64


In [39]:
a = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv")
b = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\combined1234.csv")

#csv_files=[r"C:\Users\User\PycharmProjects\sentiment\data\balanced.csv", r"C:\Users\User\PycharmProjects\sentiment\data\combined1234.csv"]
#dfsigma = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
print(a['sentiment'].value_counts())
print("aaa")
print(b['sentiment'].value_counts())

sentiment
4.0    33844
2.0    19821
0.0    12482
Name: count, dtype: int64
aaa
sentiment
4.0    33844
2.0    19821
0.0    12482
Name: count, dtype: int64


In [3]:
import pandas as pd
c = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\zbalansowanebezmondi.csv")
e = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\combined1234.csv")
g=pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\odczata.csv")


#g.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\odczata.csv",index=False)

df2_filtered = e[e["sentiment"] == 2]
df_combined = pd.concat([c, df2_filtered,g], ignore_index=True)
print(df_combined["sentiment"].value_counts())

mask = df_combined["text"].str.contains("global|warming|change|climate", case=False, na=False)

# Jeśli jest więcej niż 3000 takich wierszy, usuń losowe 3000
if mask.sum() > 3000:
    to_remove = df_combined[mask].sample(n=5000, random_state=42).index
    df_combined = df_combined.drop(index=to_remove)


min_count = df_combined["sentiment"].value_counts().min()  # 54,776

df_balanced = df_combined.groupby("sentiment").apply(lambda x: x.sample(min_count, random_state=42)).reset_index(drop=True)

print(df_balanced["sentiment"].value_counts())

df_balanced.to_csv(r"C:\Users\User\PycharmProjects\sentiment\data\zbalansowanebezprezydenta.csv",index=False)









sentiment
0.0    77637
4.0    74826
2.0    64776
Name: count, dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_3640\140299126.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_combined.groupby("sentiment").apply(lambda x: x.sample(min_count, random_state=42)).reset_index(drop=True)


sentiment
0.0    60147
2.0    60147
4.0    60147
Name: count, dtype: int64


In [6]:
from src.preprocess import  parallel_apply, process_texts, fast_clean_text
import multiprocessing as mp
import swifter
import os
e = pd.read_csv(r"C:\Users\User\PycharmProjects\sentiment\data\zbalansowanebezprezydenta.csv")


e["text"] = e["text"].fillna("").astype(str)


num_cores=mp.cpu_count() // 2
e = parallel_apply(e,process_texts, num_cores)
#save_path = os.path.join(os.getcwd(), "data", "clean_tweets.csv")

#df_combined = parallel_apply(df_combined, process_texts)
e["text"] = e["text"].apply(fast_clean_text)

e = e[e["text"].str.strip() != ""]

e.to_csv(r"C:\Users\User\PycharmProjects\sentiment\finaldata\przefiltrowanebezprezydentapowinnobycokdlaprostych.csv", index=False)

print(e["text"].dtype)
print(e["text"].head(10))  # Wyświetl pierwsze wartości w kolumnie "text"
print(e["sentiment"].value_counts())

object
0                                  hug need hurt anyone
1                             evening shift till oclock
2                                          urgh doctors
3                                           oh stressed
5     pt awesomeclasses dayfinishing report gymmiss ...
6               wish sold time let know would love hear
7     played pokemon weekend ignored twitter totally...
8                                        going miss job
9                                            meim going
10                   hard watch seems incredibly unfair
Name: text, dtype: object
sentiment
0.0    59796
4.0    59740
2.0    59710
Name: count, dtype: int64
